### 크롤링

#### 인스타그램의 검색 결과 URL을 만드는 함수

In [1]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

#### 인스타그램에 로그인하는 함수

In [2]:
def insta_login(email,password):
    input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
    input_id.clear()
    input_id.send_keys(email)
    
    input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
    input_pw.clear()
    input_pw.send_keys(password)
    input_pw.submit()
    time.sleep(3)
    
    save_info = driver.find_element_by_css_selector('#react-root > section > main > div > div > div > section > div > button')
    save_info.click()
    time.sleep(3)
    
    alarm = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm')
    alarm.click()

#### 첫번째 게시글 열기

In [3]:
def select_first(driver):
    
    # find_element_by_css_selector : BeutifulSoup을 사용하지 않고 Selenium에서 직접 html에 접근하는 메쏘드
    first = driver.find_element_by_css_selector('div._9AhH0')
    
    # 게시물 클릭
    first.click()
    
    # 정보 로딩에 시간이 필요하므로 3초간 대기
    time.sleep(3)

#### 게시글 정보 가져오기

In [4]:
# 정규 표현식을 위한 클래스
import re

def get_content(driver):
    # 1. 현재 페이지의 HTML정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 2. 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
        
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 4. 작성일자 정보 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]
    
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like = 0
        
    # 6. 위치정보 가져오기
    try:
        place = soup.select('div.JF9hh > a.O4GlU')[0].text
    except:
        place = ''
    
    # 7. 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

In [5]:
# 다음 게시글을 여는 함수

def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

### 크롤링 과정 통합 함수

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬브라우저 열기
driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')

# 2. 인스타그램에 접속
driver.get('https://www.instagram.com')
time.sleep(2)

# 3. 인스타그램 로그인
email = '01038067982'
pw = '!@nam9595'
insta_login(email,pw)

# 4. 인스타그램 검색페이지 URL 만들기
word = '제주여행' # 검색어
url = insta_searching(word)

# 5. 검색페이지 접속하기
driver.get(url)
time.sleep(3)

# 6. 첫 번째 게시글 열기
select_first(driver)

# 7. 비어있는 변수(results) 만들기
results = []

# 8. 여러 게시물 수집하기
target = 100
for i in range(target):
    data = get_content(driver)
    results.append(data)
    move_next(driver)
    
print(results[:2])

[['.가끔씩 찾아오는 고비 같은 것들이 있다. 계획이 무너지고, 지체가 되며 가진 자원이 소모가 될때. 김영하 작가는 주인공 캐릭터의 세가지 요소를 이렇게 말했다..1. 고통을 받는다2. 분명한 목표가 있다3. 일생에 중요한 기회가 주어진다.우리는 다 각자 인생의 주인공이니까 견디고, 목적을 향해 나가며, 기회를 잘 포착해야겠다. 그런데 주인공으로 사는건 참 피곤한 일이다.나한테 하는 말이기도 하고 너한테 하는 말이기도 해:)', '2020-07-10', '1,823', '백약이 오름', []], ['여름바다가 이렇게 좋을 줄이야', '2020-07-09', '637', 'Woljeongri Beach', []]]


### 수집 데이터 저장

In [10]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'data', 'like', 'place', 'tags']
results_df.to_excel('crawling_jejutravel.xlsx')

### 여러 엑셀파일의 중복을 제거한 후 통합 저장

In [30]:
# 여러 엑셀파일을 통합
jeju_insta_df = pd.DataFrame([])

folder = 'crawling_data/'
f_list = ['crawling_jeju.xlsx', 'crawling_jejumat.xlsx', 'crawling_jejutravel.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath, usecols='B:F')
    jeju_insta_df = jeju_insta_df.append(temp)

In [35]:
jeju_insta_df.reset_index(drop=True, inplace=True)
jeju_insta_df.head()

content        data   like place  \
0  올여름은 제주도로 고고~제주갈치왕은 필수😍-여기갈치 요리가 최고로 맛있엉👍-제주산 ...  2020-07-09      0   NaN   
1  중문맛집 중문수원음식점.통갈치와 통문어 전복 새우의 환상 조합!돌솥밥과 함께라면 더...  2020-07-10  2,358   NaN   
2  #제주도여행 #성산일출봉맛집이건 꼭 먹어야 하는 흑돼지박스 도시락!!.#흑돼지박스성...  2020-07-08      0   NaN   
3  #제주여행 필수코스 #흑돼지박스 포장하고 바로옆 섭지해변에서 성산일출봉을 바라보며 ...  2020-07-10      0   NaN   
4  혼밥,혼술 또는 커플💑 가족👪모두가 어울리는 멋진 식당!!사장님이 정성껏 맛있게 식...  2020-07-09      0   NaN   

                                                tags  
0  ['#제주도맛집', '#갈치왕', '#제주도맛집베스트', '#제주도맛집추천', '#...  
1  ['#제주도맛집', '#중문', '#제주중문맛집', '#제주도갈치조림', '#제주도...  
2  ['#제주도여행', '#성산일출봉맛집이건', '#흑돼지박스성산점👉성산읍', '#섭지...  
3                    ['#제주여행', '#흑돼지박스', '#성산일출봉맛집']  
4  ['#섭지코지한끼⠀', '#제주맛집', '#제주도맛집', '#섭지코지맛집', '#성...

In [37]:
# 중복데이터 삭제

jeju_insta_df.drop_duplicates(subset = ['content'], inplace = True)
print(jeju_insta_df.shape)

jeju_insta_df.to_excel('crawling_data/crawling_raw.xlsx', index = False)

(290, 5)
